In [ ]:
!pip install pyspark datasets -q

## **train/test set pipeline**

In [ ]:
#  SETUP ===
!pip install -q pandas pyarrow scikit-learn huggingface_hub

import pandas as pd
import numpy as np
from huggingface_hub import hf_hub_download
import pyarrow.parquet as pq
import json
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
import os

#  CONFIG ===
REPO_ID = "tahoebio/Tahoe-100M"
TARGET_CELL_LINES = {
    "CVCL_0131", "CVCL_0021", "CVCL_0022",
    "CVCL_S471", "CVCL_5735", "CVCL_1239", "CVCL_RU29"
}
SHARD_RANGE = range(0, 50)        # train-00000 to train-00009
MAX_TOTAL = 50000                 # Adjust based on RAM
RAW_PATH = "tahoe_filtered_combined.jsonl"
CLEAN_PATH = "tahoe_filtered_cleaned.jsonl"
FINAL_CSV = "tahoe_model_ready.csv"

#  CLEANING HELPER ===
def make_json_serializable(obj):
    if isinstance(obj, dict):
        return {k: make_json_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple, np.ndarray)):
        return [make_json_serializable(x) for x in obj]
    elif isinstance(obj, (np.integer, np.floating)):
        return obj.item()
    elif isinstance(obj, (np.bool_)):
        return bool(obj)
    else:
        return obj

#  STEP 1: DOWNLOAD + FILTER MULTIPLE SHARDS ===
print("Downloading and filtering...")
saved = 0
with open(RAW_PATH, "w") as f:
    for i in SHARD_RANGE:
        shard_name = f"data/train-{i:05d}-of-03388.parquet"
        shard_path = hf_hub_download(repo_id=REPO_ID, filename=shard_name, repo_type="dataset")
        df = pd.read_parquet(shard_path)

        filtered = df[df["cell_line_id"].isin(TARGET_CELL_LINES)]
        for _, row in filtered.iterrows():
            row_dict = make_json_serializable(row.to_dict())
            f.write(json.dumps(row_dict) + "\n")
            saved += 1
            if saved >= MAX_TOTAL:
                break
        if saved >= MAX_TOTAL:
            break
print(f"Saved {saved} samples to {RAW_PATH}")

# === STEP 2: CLEAN THE JSON ===
print("Cleaning JSONL...")
with open(RAW_PATH, "r") as f:
    lines = f.readlines()

with open(CLEAN_PATH, "w") as f:
    for line in lines:
        try:
            obj = json.loads(line)
            cleaned = make_json_serializable(obj)
            f.write(json.dumps(cleaned) + "\n")
        except Exception as e:
            print(f" Skipping bad line: {e}")
print(f"Cleaned JSONL saved to {CLEAN_PATH}")

# === STEP 3: PREPROCESSING ===
print("Preprocessing...")
df = pd.read_json(CLEAN_PATH, lines=True)

# Feature engineering
df["expressions_length"] = df["expressions"].apply(lambda x: len(x) if isinstance(x, list) else 0)
df["mean_expression"] = df["expressions"].apply(
    lambda x: np.mean(x) if isinstance(x, list) and len(x) > 0 else 0
)
# DMSO_TF baseline
dmso_means = (
    df[df["drug"] == "DMSO_TF"]
    .groupby("cell_line_id")["mean_expression"]
    .mean()
    .rename("baseline")
)
df = df.join(dmso_means, on="cell_line_id")
df["delta_mean"] = df["mean_expression"] - df["baseline"].fillna(0)

# One-hot encode top drugs and cell lines
top_drugs = df["drug"].value_counts().nlargest(10).index.tolist()
top_cells = df["cell_line_id"].value_counts().nlargest(7).index.tolist()

df["drug_encoded"] = df["drug"].apply(lambda x: x if x in top_drugs else "Other")
df["cell_line_encoded"] = df["cell_line_id"].apply(lambda x: x if x in top_cells else "Other")

encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df[["drug_encoded", "cell_line_encoded"]])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(), index=df.index)

# Scale numeric features
scaler = StandardScaler()
numeric_cols = ["mean_expression", "delta_mean", "expressions_length"]
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Polynomial expansion
poly = PolynomialFeatures(degree=2, include_bias=False)
expanded = poly.fit_transform(df[["mean_expression", "delta_mean"]])
expanded_df = pd.DataFrame(expanded, columns=poly.get_feature_names_out(), index=df.index)

# Combine final output
final_df = pd.concat([df[numeric_cols], encoded_df, expanded_df], axis=1)
final_df.to_csv(FINAL_CSV, index=False)
print(f"Preprocessing complete. Final dataset with {final_df.shape[0]} rows and {final_df.shape[1]} features saved to `{FINAL_CSV}`")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train-00000-of-03388.parquet:   0%|          | 0.00/71.2M [00:00<?, ?B/s]

train-00001-of-03388.parquet:   0%|          | 0.00/75.0M [00:00<?, ?B/s]

train-00002-of-03388.parquet:   0%|          | 0.00/75.2M [00:00<?, ?B/s]

train-00003-of-03388.parquet:   0%|          | 0.00/78.6M [00:00<?, ?B/s]

...

train-00040-of-03388.parquet:   0%|          | 0.00/85.8M [00:00<?, ?B/s]

train-00041-of-03388.parquet:   0%|          | 0.00/89.8M [00:00<?, ?B/s]

train-00042-of-03388.parquet:   0%|          | 0.00/90.6M [00:00<?, ?B/s]

train-00043-of-03388.parquet:   0%|          | 0.00/89.9M [00:00<?, ?B/s]

Saved 50000 samples to tahoe_filtered_combined.jsonl
Cleaning JSONL...
Cleaned JSONL saved to tahoe_filtered_cleaned.jsonl
Preprocessing...
Preprocessing complete. Final dataset with 50000 rows and 21 features saved to `tahoe_model_ready.csv`


**Pipeline Overview**

1. Selective Streaming from the Tahoe-100M

dataset (using direct Parquet access from Hugging Face)

2. Filtering by cell_line_id to include only relevant glioblastoma/neuroglioma lines:

CVCL_0131, CVCL_0021, CVCL_0022, CVCL_S471, CVCL_5735, CVCL_1239, CVCL_RU29

3. Serializing and cleaning to produce a valid .jsonl file

4. Feature Engineering:

- Mean gene expression per sample

- Expression length (number of expressed genes)

- Δ mean expression relative to DMSO_TF control

5. Categorical Encoding:

- One-hot encoding of top 10 drugs and top 7 cell lines (though it looks like it is only picking up the first two found)

6. Scaling of numeric features

7. Polynomial Feature Expansion (2nd-order interactions between expression features)

8. Export to a model-ready .csv file for downstream ML tasks

**Outputs**
- tahoe_filtered_cleaned.jsonl — cleaned, valid JSON lines with full metadata

- tahoe_model_ready.csv — preprocessed, feature-expanded tabular file suitable for:

    - Classification (e.g. cell line or drug prediction)

    - Regression (e.g. predicting Δ expression)

    - Clustering or embedding (e.g. t-SNE/UMAP)



### 1. Drug × Cell Line → Δ Mean Expression
- **Task:** Regression  
- **Target Variable:** `delta_mean`  
- **Features:**
  - One-hot encoded drug  
  - One-hot encoded cell line  
- **Use Case:** Predict how strongly a drug perturbs a specific cell line's gene expression  
- Personalized drug response modeling


---

### 2. Drug × Gene Expression → Cell Line Identity
- **Task:** Multiclass Classification  
- **Target Variable:** `cell_line_encoded`  
- **Features:**
  - One-hot drug  
  - Summary of expression (`mean_expression`, `delta_mean`, `expressions_length`)  
- **Use Case:** Reconstruct cell type based on drug effect  
- Good for reverse-engineering phenotype from downstream expression

---

### 3. Drug × Cell Line → Mechanism of Action (MoA)
- **Task:** Multiclass Classification  
- **Target Variable:** `moa-fine`  
- **Features:**
  - One-hot encoded drug  
  - One-hot encoded cell line  
- **Use Case:** Infer a drug's mechanism based on where and how it acts  
- Good if data has well-annotated `moa-fine` values

---

### Drug × Cell Line × Expression → Expression Vector
- **Task:** Deep Regression (or Embedding Prediction)  
- **Target Variable:** Full expression vector (`expressions`)  
- **Features:**
  - One-hot drug  
  - One-hot cell line  
  - Optionally: `mean_expression`, `delta_mean`  
- **Use Case:** Predict full transcriptional response given a drug–cell pair  
- Used in generative models and compound screening

---

### 5. Dimensionality Reduction + Visualization
- **Task:** t-SNE or UMAP  
- **Use Case:** Visualize expression space and treatment overlap  
- Helpful for intuitive understanding of high-dimensional structure

---
What we will do next probably:
- A sample model script (e.g., RandomForest or XGBoost)
- Evaluation metrics (accuracy, F1)
- A visualization (confusion matrix or t-SNE)



## Training models

In [ ]:
import pandas as pd
import numpy as np
from huggingface_hub import hf_hub_download
import pyarrow.parquet as pq
import json
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    mean_squared_error, r2_score,
    accuracy_score, classification_report
)


In [ ]:
df = pd.read_csv('/content/tahoe_model_ready.csv')

In [ ]:
df.head()

,mean_expression,delta_mean,expressions_length,drug_encoded_(R)-Verapamil (hydrochloride),drug_encoded_Adagrasib,drug_encoded_Clonidine (hydrochloride),drug_encoded_DMSO_TF,drug_encoded_Fusidic acid,drug_encoded_Goserelin (acetate),drug_encoded_Other,...,drug_encoded_Quinestrol,drug_encoded_Selinexor,drug_encoded_Terfenadine,cell_line_encoded_CVCL_0131,cell_line_encoded_CVCL_1239,mean_expression.1,delta_mean.1,mean_expression^2,mean_expression delta_mean,delta_mean^2
0,-0.749481,-0.880731,-1.233035,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,-0.749481,-0.880731,0.561722,0.660091,0.775687
1,-1.157585,-0.800572,-1.077581,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,-1.157585,-0.800572,1.340003,0.926730,0.640915
2,1.562296,1.498096,1.828074,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.562296,1.498096,2.440768,2.340469,2.244292
3,-1.381155,-1.530725,-0.405864,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,-1.381155,-1.530725,1.907589,2.114168,2.343118
4,-0.402104,-0.523278,0.214035,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,-0.402104,-0.523278,0.161688,0.210412,0.273820


In [ ]:
df.head(2).T

,0,1
mean_expression,-0.749481,-1.157585
delta_mean,-0.880731,-0.800572
expressions_length,-1.233035,-1.077581
drug_encoded_(R)-Verapamil (hydrochloride),0.000000,0.000000
drug_encoded_Adagrasib,0.000000,0.000000
drug_encoded_Clonidine (hydrochloride),0.000000,0.000000
drug_encoded_DMSO_TF,0.000000,0.000000
drug_encoded_Fusidic acid,0.000000,0.000000
drug_encoded_Goserelin (acetate),0.000000,0.000000
drug_encoded_Other,1.000000,1.000000


## 1. Drug × Cell Line → Δ Mean Expression Task: Regression Target Variable: delta_mean Features: One-hot encoded drug One-hot encoded cell line Use Case: Predict how strongly a drug perturbs a specific cell line's gene expression Personalized drug response modeling

In [ ]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# 1) Load training data
train = pd.read_csv("/content/tahoe_model_ready.csv")
feature_cols = [c for c in train.columns if c.startswith("drug_encoded_") or c.startswith("cell_line_encoded_")]
X_train = train[feature_cols]
y_train = train["delta_mean"]

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train Ridge model
model_ridge = Ridge(alpha=1.0)
model_ridge.fit(X_train_r, y_train_r)

# Predict
y_train_pred_r = model_ridge.predict(X_train_r)
y_test_pred_r = model_ridge.predict(X_test_r)

In [ ]:
print("=== Ridge Regression ===")
print(f"Train MSE: {mean_squared_error(y_train_r, y_train_pred_r):.4f}")
print(f"Test MSE:  {mean_squared_error(y_test_r, y_test_pred_r):.4f}")
print(f"Train R2:  {r2_score(y_train_r, y_train_pred_r):.4f}")
print(f"Test R2:   {r2_score(y_test_r, y_test_pred_r):.4f}")

=== Ridge Regression ===
Train MSE: 0.9863
Test MSE:  0.9631
Train R2:  0.0183
Test R2:   0.0185


This code trains a Ridge Regression model to predict the delta_mean gene expression changes using one-hot encoded drug and cell line features from the tahoe_model_ready.csv dataset. After splitting the data into training and test sets, the model shows similar performance across both: Train MSE = 0.9863 and Test MSE = 0.9631, with very low R² scores (Train R² = 0.0183, Test R² = 0.0185). This indicates the model generalizes reasonably but explains less than 2% of the variance, suggesting that the chosen features have limited predictive power for this task.

## 2. Drug × Gene Expression → Cell Line Identity Task: Multiclass Classification Target Variable: cell_line_encoded Features: One-hot drug Summary of expression (mean_expression, delta_mean, expressions_length) Use Case: Reconstruct cell type based on drug effect Good for reverse-engineering phenotype from downstream expression

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder


df = pd.read_csv("tahoe_model_ready.csv")

drug_cols = [c for c in df.columns if c.startswith("drug_encoded_")]
expr_cols = ["mean_expression", "delta_mean", "expressions_length"]
cell_cols = [c for c in df.columns if c.startswith("cell_line_encoded_")]


cell_labels = df[cell_cols].idxmax(axis=1)
cell_labels = cell_labels.str.replace("cell_line_encoded_", "", regex=False)
le = LabelEncoder()
y = le.fit_transform(cell_labels)

X = df[drug_cols + expr_cols]

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
clf_rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
clf_rf.fit(X_train_c, y_train_c)

# Predict
y_train_pred_c = clf_rf.predict(X_train_c)
y_test_pred_c = clf_rf.predict(X_test_c)

# Evaluate
print("\n=== Random Forest Classifier ===")
print(f"Train Accuracy: {accuracy_score(y_train_c, y_train_pred_c):.4f}")
print(f"Test Accuracy:  {accuracy_score(y_test_c, y_test_pred_c):.4f}")

print("\nClassification Report (Test):")
print(classification_report(y_test_c, y_test_pred_c, target_names=le.classes_))


=== Random Forest Classifier ===
Train Accuracy: 1.0000
Test Accuracy:  0.9974

Classification Report (Test):
              precision    recall  f1-score   support

   CVCL_0131       1.00      1.00      1.00      7818
   CVCL_1239       1.00      0.99      0.99      2182

    accuracy                           1.00     10000
   macro avg       1.00      0.99      1.00     10000
weighted avg       1.00      1.00      1.00     10000



This code trains a Random Forest Classifier to predict the cell line identity from drug encoding features and expression-related features. The target labels are extracted by identifying the one-hot encoded cell_line_encoded_* columns with the highest value per row and then converting them to numeric labels using LabelEncoder. The input features include drug encodings and expression metrics like mean_expression, delta_mean, and expressions_length. After splitting the data into training and test sets, the classifier is trained and evaluated. The model achieves perfect training accuracy (1.0000) and nearly perfect test accuracy (0.9974), with precision, recall, and F1-scores all close to or equal to 1.00 for both classes.

This high level of performance suggests that the model both fits and generalizes extremely well to this classification task, indicating that the input features are highly informative for predicting the cell line identity. Unlike the Ridge Regression model (which explained less than 2% of the variance in a regression task), this Random Forest classifier performs exceptionally well in its classification objective.

Our Random Forest model fits in the ideal region of the bias-variance tradeoff, as our training accuracy is perfect (1.0) and our test accuracy is nearly perfect (0.9974). The extremely small gap of 0.26% between the training and testing sets indicates that the model has learned underlying patterns without significant overfitting. However, the perfect training accuracy raises a small red flag as the model may be memorizing some of the training data. Though, the high testing performance suggests that the model was still able to generalize successfully. This can suggest that cell line identification from drug/expression features could have clear distinguishable patterns.

Although our perfect training accuracy is not inherently alarming, it is still worth investigating, as we can ensure robustness when the model is deployed on completely new data. To further improve performance, we can try other ensemble methods that could capture different patterns or insights through gradient boosting. We can implement XGBoost and LightGBM, which uses sequential error correction, rather than Random Forest's parallel tree averaging. By building on our current model, the new model can correct the errors within our gap of 0.26% and can provide faster runtime. We can also cross validate with different Random Forests configurations to confirm consistent performance. By experimenting with the number of trees, max_depth, and min_samples_split, we can reduce the risk of overfitting, while improving its generalization on unseen data.  